In [ ]:
import os
import xarray as xr

# Directory where your files are stored
data_dir = '../Data/ALAN'

# List all files in the directory
all_files = [f for f in os.listdir(data_dir) if f.endswith('.nc')]

# Group files by month based on the filename pattern `Month-XX`
monthly_files = {}
for file in all_files:
    # Extract month from the file name
    month = file.split('_')[4].split('-')[1]  # Assuming the format `Month-XX`
    if month not in monthly_files:
        monthly_files[month] = []
    monthly_files[month].append(os.path.join(data_dir, file))

# Merge files by month and save as NetCDF
for month, files in monthly_files.items():
    datasets = [xr.open_dataset(f) for f in files]
    combined = xr.concat(datasets, dim='time')  # Adjust 'dim' as needed based on your data structure
    combined_file = os.path.join(data_dir, f"combined_month_{month}.nc")
    combined.to_netcdf(combined_file)
    print(f"Saved combined file for Month-{month} as {combined_file}")


In [10]:
import xarray as xr
import os
import glob
import numpy as np

data_dir = "../Data/ALAN/"
file_pattern = "In-water_clear-sky_ALAN_Zc_Month-{month:02d}_*.nc"

merged_data = {}

for month in range(1, 13):
    print(month)
    month_files = glob.glob(os.path.join(data_dir, file_pattern.format(month=month)))
    
    datasets = []
    print(month)
    for file in month_files:
        ds = xr.open_dataset(file)
        
        # Optionally drop duplicates if they exist
        ds = ds.drop_duplicates(dim='lat').drop_duplicates(dim='lon')
        
        datasets.append(ds)
    print(month)
    # Combine datasets for the month across lat/lon dimensions
    merged_data[month] = xr.combine_by_coords(datasets, combine_attrs="drop_conflicts")

# Optionally save the merged datasets
for month in merged_data:
    output_path = os.path.join(data_dir, f"ALAN_Zc_Month-{month:02d}_merged.nc")
    merged_data[month].to_netcdf(output_path)


1
1
1


ValueError: cannot reindex or align along dimension 'lat' because the (pandas) index has duplicate values